In [1]:
from flair.data import Sentence
from flair.nn import Classifier
import pandas as pd
import re
import nltk
import string
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matthewschultz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/matthewschultz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
tweets = pd.read_csv('/Users/matthewschultz/Big_Data_Lab/dominion_voting/output/attempt.csv', low_memory=False)

In [3]:
def remove_punctuation(x):
  new_list = [re.sub("/\\n|\,|\(|\)|\[|\]/g", "", word) for word in x]
  i_list = [re.split("\/", word) for word in new_list]
  flatten_list = lambda irregular_list:[element for item in irregular_list for element in flatten_list(item)] if type(irregular_list) is list else [irregular_list]
  list4 = flatten_list(i_list)
  final_list = " ".join([word for word in list4 if word != ""])
  final_list = final_list.translate(str.maketrans('', '', string.punctuation))
  return final_list


def clean_tweets(df):
    list_of_tweets = []
    df = df.dropna(subset=['Title'])
    for index, row in df.iterrows():
        print(index)
        tweet = row['Title']
        
        tweet = re.sub(r'https?:\/\/.\S+', "", tweet)
        tweet = re.sub(r'#', '', tweet)
        tweet = re.sub(r'^RT[\s]+', '', tweet)
        tweet = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",tweet) if s])

        tweet = nltk.word_tokenize(tweet)
        tweet = remove_punctuation(tweet)
        tweet = re.sub(" +", " ", tweet)
        list_of_tweets.append(tweet)
        if index == 20:
           return list_of_tweets
        

cleaned_tweets = clean_tweets(tweets)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [4]:
def create_ner_df(lis):
    entity_ = []
    tag_ =  []
    index = 0
    for tweet in lis:
        print(index)
        sentence = Sentence(tweet)

        # load the NER tagger
        tagger = Classifier.load('ner')

        # run NER over sentence
        tagger.predict(sentence)
        for label in sentence.get_labels():
            entity_.append(label.data_point.text)
            tag_.append(label.value)
        index = index + 1

    final = pd.DataFrame({'entity': entity_, 'tag': tag_})
    return final

ner_df = create_ner_df(cleaned_tweets)



0
2023-04-14 10:19:45,147 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
1
2023-04-14 10:19:51,387 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
2
2023-04-14 10:19:57,477 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
3
2023-04-14 10:20:03,357 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
4
2023-04-14 10:20:12,074 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG,

In [5]:
freq_table_frequency = ner_df.groupby(['entity', 'tag']).size().reset_index(name="frequency")

print(freq_table_frequency)

                                               entity   tag  frequency
0                                              Bowers   PER          1
1                                               CHINA   LOC          1
2                                                 CNN   ORG          1
3                                Carlos Simancas Raul   PER          1
4                                            Congress   ORG          4
5                                           Democrats  MISC          4
6                                              Denver   LOC          1
7                                            Dominion   ORG          5
8                            Dominion Voting Services   ORG          1
9                              Dominion Voting System  MISC          2
10                            Dominion Voting Systems   ORG         10
11        Dominion Voting Systems Lead To Data Center   ORG          1
12                               Dominion Voting scam  MISC          1
13    